## [단계 ③] 학습, 테스트 데이터 선택
③-1. 학습, 테스트 데이터 선택
- 학습 데이터로는 길이가 적당한 정상적인 데이터로서 배터리 충전 데이터 중 하나인“1000_chg.csv”을 택한다.
- 이상 감지 테스트를 위하여 불량이 있는 테스트 데이터를 선택하여 MTadGAN 테스트를 수행한다. 본 가이드북에서 사용될 테스트 데이터 파일은 “Test07_NG_dchg.csv”인데 이 데이터는 센서와이어 불량으로 인해 전압 이상을 보인다.

## [단계 ④] AI모델 알고리즘 선택
④-1. AI모델 알고리즘 선택
- AI모델 알고리즘으로는 기존의 TadGAN 방법을 다변량 데이터로 확장한 MTadGAN 알고리즘을 주된 알고리즘으로 택하고 데이터의 차원을 축소하기 위하여 PCA 알고리즘을 초기 데이터 처리와 결합한다. MTadGAN 알고리즘을 사용하기 위해서는 Tensorflow 패키지를 설치하여야한다. 
- Tensorflow 및 분석에 필요한 패키지를 설치한다. 

In [ ]:
# !pip install tensorflow==2.5.0
# !pip install graphviz==0.20.1
# !pip install pydot==1.4.2
# !pip install pydotplus==2.0.2
# !pip install pyts==0.12.0

- 데이터에 대한 학습과 테스트에 필요한 라이브러리를 호출한다. 대표적인 라이브러리로는 tensorflow 이외에 데이터를 편리하게 다루기 위한 라이브러리로서 pandas, numpy가 있고 그래픽 도구로서는 plotly, matplotlib 등이 있으며 다양한 기계학습 기타 통계 계산 도구로서 sklearn(scikit-learn)이 있다.
- pandas: 이차원의 데이터를 이름이 부여된 칼럼별로 테이블 형식으로 저장하고 편리하게 조작할 수 있도록 해준다.
- numpy: 다차원의 배열로 이루어진 수치 데이터를 편리하게 다룰 수 있도록 한다.

In [1]:
import os
import sys
import numpy as np
import pandas as pd
# 라이브러리에서 필요한 모듈만 호출
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.impute import SimpleImputer
from datetime import datetime
import plotly.graph_objects as go
import plotly.express as px
import math
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from random import randrange
import argparse
import collections
import tensorflow as tf
import logging
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Bidirectional, LSTM, Flatten, Dense, Reshape, UpSampling1D, TimeDistributed
from tensorflow.keras.layers import Activation, Conv1D, LeakyReLU, Dropout, Add, Layer
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.optimizers import Adam
import pydot
import pydotplus
from pydotplus import graphviz
from scipy import stats
from scipy import integrate
from scipy.optimize import fmin
from pyts.metrics import dtw
from pandas.plotting import register_matplotlib_converters

## [단계 ⑤] AI모델 학습
⑤-1. AI모델 학습
- 먼저 MTadGAN을 이용하기 위해 필요한 파라미터를 설정한다. 중요한 파라미터로는 win_size, features_dim이 있는데 win_size 파라미터는 시계열 데이터에서 MTadGAN 모형으로 입력하는 데이터의 시간 윈도우의 크기를 나타낸다. 본 가이드북에서는 win_size=10으로 설정한다. 한편 features_dim 파라미터는 MTadGAN 모형에서 받아들이는 데이터 피쳐의 차원수를 말하는데 여기서는 features_dim=3으로 설정한다. 따라서 원래 데이터의 개수(차원수)는 약 200 쯤 되므로 PCA 알고리즘을 써서 이를 3 차원으로 축소한다. 이에 더하여 Dictionary 자료형 변수로 params를 정의하여 여러 가지 파라미터를 정의하는데 예를 들면 params[‘epochs’] 은 MTadGAN 모형 학습(training)에서의 반복 횟수를 나타내고 params[‘learning_rate’]는 MTadGAN 모형 학습에서의 학습률을 나타내며 params[‘latent_dim’]는 입력 데이터 벡터를 부호화(encoding) 할 때의 숨은(잠재) 공간의 차원을 표시한다.

In [2]:
# hyper parameters
win_size =10                        # 시간의 윈도우 크기
features_dim =3                     # PCA의 차원 수
feat_dim = features_dim
params = {}
params['plot_network'] =True        # 네트워크 구조를 시각화할지 여부를 나타내는 플래그.
params['epochs'] =30                # 학습 시 전체 데이터셋을 몇 번 반복할지를 나타내는 에폭 수.
params['batch_size'] =64            # 한 번의 학습에 사용되는 데이터 샘플의 수를 나타내는 배치 크기
params['n_critic'] =5               # 비판자(critic) 네트워크의 업데이트 횟수
params['learning_rate'] =0.00005    # 모델 학습 시 가중치를 업데이트하는 학습률
params['latent_dim'] =20            # 잠재 공간(latent space)의 차원 수. 일반적으로 생성 모델(GAN)에서 사용
params['shape'] = [win_size, features_dim]                  # 입력 데이터의 기본 형태
params['encoder_input_shape'] = [win_size, features_dim]    # 인코더 네트워크의 입력 형태
params['encoder_reshape_shape'] = [20, 1]                   # 인코더 네트워크의 출력 형태를 변환할 때의 목표 형태
params['generator_input_shape'] = [20, 1]                   # 생성기 네트워크의 입력 형태
params['generator_reshape_shape'] = [win_size, 1]           # 생성기 네트워크의 출력 형태를 변환할 때의 목표 형태
params['critic_x_input_shape'] = [win_size, features_dim]   # 비판자 네트워크의 X 입력 형태.
params['critic_z_input_shape'] = [20, 1]                    # 비판자 네트워크의 Z 입력 형태
print("win_size = %d, features_dim = %d " % (win_size, features_dim))

win_size = 10, features_dim = 3 


- 학습 데이터 파일 “1000_chg.csv”을 지정하여 args.signal_file이라는 변수를 저장하며 기타 변수를 정한다. 이상값의 파일이 있으면 args.anomaly_file이라는 변수에 저장한다. args.mode = ‘train’은 학습 모드임을 나타낸다. 

In [3]:
arguments=collections.namedtuple('Args',
                            'signal_file timest_form anomaly_file mode aggregate_interval regate_interval')

args=arguments(signal_file = '../../../../data/preprocessed/train/1000_chg.csv',
    timest_form = 0,
    anomaly_file = '',
    mode = 'train',
    aggregate_interval = 1,
    regate_interval = 1)
print("argments")
print("----------------------")
print("args.timest_form :", args.timest_form)
print("args.signal_file :", args.signal_file)
print("args.anomaly_file :", args.anomaly_file)
print("args.mode :", args.mode)
print("args.aggregate_interval :", args.aggregate_interval)
print("----------------------\n")

argments
----------------------
args.timest_form : 0
args.signal_file : ../../../../data/preprocessed/train/1000_chg.csv
args.anomaly_file : 
args.mode : train
args.aggregate_interval : 1
----------------------



- 학습 데이터에 대한 차분 함수 diff_smooth_df()를 정의한다. 이 함수는 입력 데이터값을 일정한 시차(diffs_n>1)로 차이값을 구하거나 인접한 시점의 데이터 값들을 결합하여 데이터를 스무드하게 만드는 역할을 한다. 아래 코드에서는 diffs_n=0 등으로 지정하므로 차분함수가 원래 데이터를 그대로 두게 된다.

### 차분
- 시계열 데이터의 분석 및 예측에서 중요한 기법 중 하나로, 데이터의 트렌드(추세)나 계절성을 제거하여 시계열 데이터를 안정화(stationary)시키는 데 사용
- 차분은 시계열 데이터의 연속적인 데이터 포인트 간의 차이를 계산하는 것
1. 1차 차분
    - 연속적인 데이터 포인트 간의 차이를 계산
    - 수식: $ \Delta y_t = y_t - y_{t-1} $
    - 주로 데이터의 단순 추세를 제거하는 데 사용
2. 2차 차분
    - 1차 차분을 한 번 더 차분
    - 수식 : $ \Delta^2 y_t = \Delta y_t - \Delta y_{t-1} = (y_t - y_{t-1}) - (y_{t-1} - y_{t-2}) = y_t - 2y_{t-1} + y_{t-2} $
    - 더 복잡한 추세나 패턴을 제거하는 데 사용
3. 계절 차분
    - 특정 계절 주기를 고려하여 차분
    - 수식 : $ \Delta_s y_t = y_t - y_{t-s} $
    - 주로 계절성을 제거하는 데 사용
4. d차 차분
    - 수식 : $ \Delta^d y_t = \sum_{k=0}^{d} (-1)^k \binom{d}{k} y_{t-k} $
#### 차분의 목적
- 시계열 데이터의 비정상성을 제거해 데이터를 정상성을 띄게 만드는 것. 정상성을 띄는 시계열 데이터는 평균, 분산이 시간에 따라 변하지 않으며, 자기상관성이 일정한 특성을 가짐. 

### 평활화
- 주로 이동 평균을 계산하는 방식으로 이루어짐. 평활화횟수는 이동 평균을 계산할 때 창의 크기를 나타냄. 예를 들어, 평활화 횟수가 3이면 각 데이터 포인트는 자신과 그 이전 2개 포인트를 포함한 3개의 데이터 포인트의 평균으로 대체.

#### 효과
- 노이즈 감소 : 평활화는 데이터의 단기적인 변동을 줄이고, 더 장기적인 추세를 강조
- 추세 시각화 : 데이터의 주요 추세를 더 명확하게 볼 수 있게 함

In [4]:
diffs_n = 0     # 차분을 적용할 횟수
lags_n = 0      # 특징 벡터에 포함될 시차의 개수
smooth_n = 0    # 특징 벡터에 포함될 최신 값들을 평활화할 횟수
print("diffs_n: ", diffs_n)

def diff_smooth_df(df, lags_n, diffs_n, smooth_n, diffs_abs=False, abs_features=False):
    """입력 데이터프레임에 대해 차분, 평활화, 시차를 적용하여 전처리"""
    # 차분 차수가 1 이상인 경우 차분 적용
    if diffs_n >= 1:
        df = df.diff(diffs_n).dropna()
    # 절댓값 차분을 적용
    if diffs_abs == True:
        df = abs(df)
    # 이동 평균을 적용하여 데이터를 약간 평활화함
    if smooth_n >= 2:
        df = df.rolling(smooth_n).mean().dropna()
    # 라그 값이 1 이상인 경우
    if lags_n >= 1:
        # 각 차원에 대해 차분 및 평활화된 값에 대해 lags_n 시차의 새로운 열을 추가함
        df_columns_new = [f'{col}_lag{n}' for n in range(lags_n + 1) for col in df.columns]
        df = pd.concat([df.shift(n) for n in range(lags_n + 1)], axis=1).dropna()
        df.columns = df_columns_new
    # 특징 벡터를 볼 때 명확성을 위해 열을 정렬함
    df = df.reindex(sorted(df.columns), axis=1)
    # 모든 특징을 절대값으로 변환함 (만약 abs_features가 True인 경우)
    if abs_features == True:
        df = abs(df)

    return df

print("import diff_smooth_df() : Done ! ")

diffs_n:  0
import diff_smooth_df() : Done ! 


- PCA 알고리즘을 적용하여 학습 데이터의 차원을 3차원으로 축소한다.

In [5]:
######## 데이터 PCA 진행 ################################
signal_path = args.signal_file
df_train_0= pd.read_csv(signal_path)
data_0 = df_train_0                 # 데이터 복사
# preprocess or 'featurize' the training data
data_1 = diff_smooth_df(data_0, lags_n, diffs_n, smooth_n)
pca = PCA(n_components=features_dim)        # 입력 차원의 PCA 생성
data = pca.fit_transform(data_1)            # 입력 데이터 PCA로 차원 축소 수행
df_1 = []                                   # 데이터프레임 역할을 할 배열
for i in range(len(data)):
    row = [i+1]                             # 각 행의 인덱스 설정
    for jj in range(features_dim):          # 차원 수 만큼 주성분 입력
        row.append(data[i][jj])
    df_1.append(row)                        # 데이터프레임에 추가
df = pd.DataFrame(df_1)                     # 반복 수행 후 데이터 프레임화
columns_new = ['date']                      # 데이터프레임의 인덱스열
for i in range(1, features_dim+1):          # pca
    pca_i = f'pca_{str(i)}'                 # 제i 주성분
    columns_new.append(pca_i)
df.columns = columns_new
print("주성분 차원 축소 후 df.shape = ", df.shape)
print("----------------------")
print(df.head(5))
print("----------------------")
print("\n")

주성분 차원 축소 후 df.shape =  (6009, 4)
----------------------
   date      pca_1     pca_2     pca_3
0     1  18.634349  0.564999  0.261410
1     2  18.635771  0.571294  0.263656
2     3  18.635892  0.570799  0.263567
3     4  18.634366  0.564929  0.261397
4     5  18.666694  0.565826  0.255594
----------------------




- 입력 학습 데이터의 시간-집합화(time-aggregate) 과정을 수행하는 함수 time_segments_aggregate()를 정의한다. 이는 위에서 정의한 변수 args.aggregate_interval의 값이 1보다 클 때 그 구간 안의 데이터를 평균하거나하여 데이터를 조정하는 함수이다. 여기에서는 args.aggregate_interval=1로 설정한다.

In [6]:
def time_segments_aggregate(X, interval, time_column, method=['mean']):
    """주어진 시간 간격에 대해 값을 집계함.
    Args:
    X (ndarray 또는 pandas.DataFrame): N차원 시퀀스 값.
    interval (int): 집계를 계산할 시간 간격을 나타내는 정수.
    time_column (int): X에서 시간 값을 포함하는 열.
    method (str 또는 list):
    선택적. 집계 방법을 설명하는 문자열 또는 여러 집계 방법을 설명하는 문자열 목록. 지정하지 않으면 'mean'이 사용.
    Returns:
    ndarray, ndarray:
    * 집계된 값의 시퀀스, 각 집계 방법에 대해 하나의 열.
    * 집계된 각 세그먼트의 첫 번째 인덱스 값의 시퀀스.
    """
    if isinstance(X, np.ndarray):   # 만약 X가 배열이라면 데이터프레임으로 변환
        X = pd.DataFrame(X)
    X = X.sort_values(time_column).set_index(time_column)   # 시간 열을 기준으로 정렬하고, 시간 열을 인덱스로 설정
    if isinstance(method, str):     # 만약 `method`가 문자열이면, 리스트로 변환
        method = [method]
    start_ts = X.index.values[0]    # 데이터의 시작 시간
    max_ts = X.index.values[-1]     # 데이터의 마지막 시간
    values = list()                 # 집계된 값을 저장할 리스트
    index = list()                  # 집계된 각 세그먼트의 시작 시간을 저장할 리스트
    while start_ts <= max_ts:       # 시작부터 마지막까지
        end_ts = start_ts + interval    # 시간 간격을 계산
        subset = X.loc[start_ts:end_ts-1]   # 해당 간격을 선택
        aggregated = [
            getattr(subset, agg)(skipna=True).values
            for agg in method]
        values.append(np.concatenate(aggregated))   # 집계 결과 추가
        index.append(start_ts)                      # 시작 시간을 리스트에 추가
        start_ts = end_ts                           # 시작 시간 갱신(현재 간격 종료 시간으로)
    return np.asarray(values), np.asarray(index)    # 집계 결과를 ndarray로 반환


- time_segments_aggregate() 함수를 작용하고 이들 값을 [-1,+1] 사이의 값으로 정규화한다.

In [7]:
# 시간 집합화 수행
X, index = time_segments_aggregate(df, interval=args.aggregate_interval, time_column='date')
print("signal data (after time_segments_aggregate) = ", X.shape)
print("--------------------------------------------")
print(X[:5])
print("--------------------------------------------")
print("\n")

# 최대최소 정규화 적용
X = SimpleImputer().fit_transform(X)
X = MinMaxScaler(feature_range=(-1, 1)).fit_transform(X)
print("X (after MinMaxScaler) = ", X.shape)
print("--------------------------------------------")
print(X[:5])
print("--------------------------------------------")
print("\n")
X_norm = X 

signal data (after time_segments_aggregate) =  (6009, 3)
--------------------------------------------
[[18.6343489   0.56499908  0.26141038]
 [18.63577092  0.57129376  0.26365645]
 [18.63589171  0.57079932  0.26356652]
 [18.63436617  0.56492876  0.26139721]
 [18.66669403  0.56582562  0.25559404]]
--------------------------------------------


X (after MinMaxScaler) =  (6009, 3)
--------------------------------------------
[[0.99721634 0.48867576 0.26892606]
 [0.99732535 0.49487412 0.27233804]
 [0.99733461 0.49438724 0.27220142]
 [0.99721766 0.48860651 0.26890604]
 [0.99969584 0.48948965 0.2600905 ]]
--------------------------------------------




- 학습 데이터를 AI 알고리즘이 요구하는 입력 윈도우 길이(win_size)에 맞게 묶는 함수인 rolling_window_sequences() 함수를 정의한다. 즉, 만일 win_size=10이면 연속한 10개의 데이터 값을 벡터로 묶어서 입력값으로 사용한다.

In [8]:
def rolling_window_sequences(X, index, window_size, target_size, step_size,
    target_column, drop=None, drop_windows=False):
    """시계열 데이터에서 롤링 윈도우 시퀀스를 생성함.
    이 함수는 입력 시퀀스를 롤링 윈도우로 순회하며 입력 시퀀스 배열과 목표 시퀀스 배열을 생성함.
    선택적으로 시퀀스에서 특정 값을 삭제할 수 있음.
    
    Args:
    X (ndarray): 순회할 N차원 시퀀스.
    index (ndarray): X의 인덱스 값을 포함하는 배열.
    window_size (int): 입력 시퀀스의 길이.
    target_size (int): 목표 시퀀스의 길이.
    step_size (int):  # 윈도우를 각 라운드마다 몇 단계씩 앞으로 이동할지 나타내는 값.
    target_column (int): X의 어느 열이 목표인지를 나타내는 값.
    drop (ndarray 또는 None 또는 str 또는 float 또는 bool):  # 선택적. X의 어떤 값이 유효하지 않은지 나타내는 부울 값 배열 또는 삭제할 값을 나타내는 값. 지정하지 않으면 `None`이 사용됨.
    drop_windows (bool): 선택적. 삭제 기능을 활성화할지 여부를 나타냄. 지정하지 않으면 `False`가 사용됨.
    
    Returns:
    ndarray, ndarray, ndarray, ndarray:
    * 입력 시퀀스.
    * 목표 시퀀스.
    * 각 입력 시퀀스의 첫 번째 인덱스 값.
    * 각 목표 시퀀스의 첫 번째 인덱스 값.
    """
    out_X = list()
    out_y = list()
    X_index = list()
    y_index = list()
    target = X[:, target_column]
    if drop_windows:
        if hasattr(drop, '__len__') and (not isinstance(drop, str)):
            if len(drop) != len(X):
                raise Exception('배열 `drop`과 `X`는 같은 길이여야 합니다.')
        else:
            if isinstance(drop, float) and np.isnan(drop):
                drop = np.isnan(X)
            else:
                drop = X == drop
    start = 0
    max_start = len(X) - window_size - target_size + 1
    while start < max_start:
        end = start + window_size
        if drop_windows:
            drop_window = drop[start:end + target_size]
            to_drop = np.where(drop_window)[0]
            if to_drop.size:
                start += to_drop[-1] + 1
                continue
        out_X.append(X[start:end])
        out_y.append(target[end:end + target_size])
        X_index.append(index[start])
        y_index.append(index[end])
        start = start + step_size
    return np.asarray(out_X), np.asarray(out_y), np.asarray(X_index), np.asarray(y_index)


- 학습 데이터에 rolling_window_sequences() 함수를 적용하여 MTadGAN 알고리즘에 입력할 수 있도록 만든다. 즉, MTadGAN에서는 시계열 데이터를 모형으로 입력할 때에 특정한 시점에 대하여 일정한 윈도우 길이(win_size)만큼 묶어서 입력하게 된다. rolling_window_sequences() 함수는 입력 윈도우 길이(win_size)에 맞게 모든 시점에 대하여 묶는 함수이다. 만일 win_size=10이면 모든 시점에 대하여 각각 근처의 연속한 10개의 데이터 값을 벡터로 묶어서 입력값으로 사용할 수 있게 한다.

In [9]:
# 윈도우 길이에 맞게 묵음
X, y, X_index, y_index=rolling_window_sequences(X, index, window_size=win_size, target_size=1, step_size=1, target_column=0)
print("X shape (after rolling_window_seq): {}".format(X.shape))
print("X : ")
print(X[:3, :5])
print("X index shape: {}".format(X_index.shape))
print("y shape: {}".format(y.shape))
print("y : ")
print(y[:5])
print("y index shape: {}".format(y_index.shape))
print("\n")

X shape (after rolling_window_seq): (5999, 10, 3)
X : 
[[[0.99721634 0.48867576 0.26892606]
  [0.99732535 0.49487412 0.27233804]
  [0.99733461 0.49438724 0.27220142]
  [0.99721766 0.48860651 0.26890604]
  [0.99969584 0.48948965 0.2600905 ]]

 [[0.99732535 0.49487412 0.27233804]
  [0.99733461 0.49438724 0.27220142]
  [0.99721766 0.48860651 0.26890604]
  [0.99969584 0.48948965 0.2600905 ]
  [0.99729103 0.49165725 0.27192521]]

 [[0.99733461 0.49438724 0.27220142]
  [0.99721766 0.48860651 0.26890604]
  [0.99969584 0.48948965 0.2600905 ]
  [0.99729103 0.49165725 0.27192521]
  [0.99741076 0.49573991 0.26903184]]]
X index shape: (5999,)
y shape: (5999, 1)
y : 
[[0.99860594]
 [0.99594234]
 [0.99452801]
 [1.        ]
 [0.9959476 ]]
y index shape: (5999,)




- GPU가 있는지 체크한다. GPU가 있을 시 GPU 정보가 나오며, GPU가 없을 시 아래와 같은 결과가 나오게 된다.

In [10]:
# GPU 환경 존재 여부 확인
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
print (gpus)

[]


- MTadGAN에 필요한 함수 중 하나로서 RandomWeightedAverage() 클래스를 정의하는데 이는 입력 데이터와 예측된 입력 데이터를 정해진 비율로 선형 결합하는 역할을 한다.

In [11]:
class RandomWeightedAverage(Layer):
    def __init__(self, batch_size):
        """레이어 초기화
        Args:
        batch_size: 배치 크기 (예: 64)
        """
        super().__init__()
        self.batch_size = batch_size

    def call(self, inputs, **kwargs):
        """랜덤 가중 평균 계산
        Args:
        inputs[0] x: 원래 입력
        inputs[1] x_: 예측된 입력
        """
        alpha = K.random_uniform((self.batch_size, 1, 1))  # 배치 크기에 따라 랜덤 가중치 생성
        return (alpha * inputs[0]) + ((1 - alpha) * inputs[1])  # 원래 입력과 예측된 입력의 랜덤 가중 평균을 반환

print("RandomWeightedAverage() Class defined, Done! ") 

RandomWeightedAverage() Class defined, Done! 


- MTadGAN의 encoder 레이어를 생성하는 함수를 정의한다. 이는 입력 데이터를 숨은 공간으로 임베딩하는 기능을 한다.

In [12]:
def build_encoder_layer(input_shape, encoder_reshape_shape):
    """인코더 레이어를 생성
    인자들:
    input_shape: [10, 1]
    encoder_reshape_shape: [20, 1]
    
    Returns:
    인코더 모델
    """
    x = Input(shape=input_shape)
    model = tf.keras.models.Sequential([
        Bidirectional(LSTM(units=win_size, return_sequences=True)), 
        Flatten(),
        Dense(20), # 20 = self.critic_z_input_shape[0]
        Reshape(target_shape=encoder_reshape_shape)]) # (20, 1)
    return Model(x, model(x))

print("인코더 레이어 생성 함수 정의 완료")

인코더 레이어 생성 함수 정의 완료


- Bidirectional : LSTM 레이어를 감싸서 순방향과 역방향 모두 입력 시퀀스를 처리하도록 함
    - units : LSTM 레이어의 유닛(뉴런) 수. win_size 만큼의 unit을 가짐
    - return_sequences = True : 각 타임스텝마다 생성된 출력 벡터 반환
    - units 인자를 증가시키는 경우 더 복잡한 학습이 가능하지만 과적합의 위험이 커짐
- Flatten : 다차원 텐서 배열을 차원으로 변환시키는 역할
- Dense : 완전 연결층. 모든 입력 뉴런이 모든 출력 뉴런과 연결되도록 함.
    - units : 뉴런의 수. 여기서 
    - activation : 활성화 함수. 미지정 시 linear(선형)
- Reshape : 입력 데이터의 차원을 변경하는 역할
- 모델의 층
```
Input Layer: (10, 1)
    |
Bidirectional LSTM Layer (units=win_size): (10, 2 * win_size)
    |
Flatten Layer: (10 * 2 * win_size)
    |
Dense Layer (units=20): (20)
    |
Reshape Layer (target_shape=(20, 1)): (20, 1) : 출력층의 역할
```

- MTadGAN의 generator 레이어를 생성하는 함수를 정의한다. 이는 숨은 공간으로부터 가상의 데이터를 생성하는 역할을 한다.

In [13]:
def build_generator_layer(input_shape, generator_reshape_shape):
    # input_shape = (20, 1) / generator_reshape_shape = (50, 1)
    x = Input(shape=input_shape)
    model = tf.keras.models.Sequential([
        Flatten(),  # 입력을 1차원으로 평탄화
        Dense(win_size),  # 원래 50이었던 win_size만큼의 유닛을 가진 Dense 레이어
        Reshape(target_shape=generator_reshape_shape),  # (50, 1) 형태로 변환
        # 양방향 LSTM 레이어, 64 유닛, 시퀀스를 반환하며, 순방향과 역방향 출력을 연결
        Bidirectional(LSTM(units=64, return_sequences=True), merge_mode='concat'),  
        Dropout(rate=0.2),  # 드롭아웃 20%
        #UpSampling1D(size=2),
        UpSampling1D(size=1),  # 시퀀스 데이터를 1배 업샘플링
        # 양방향 LSTM 레이어, 64 유닛, 시퀀스를 반환하며, 순방향과 역방향 출력을 연결
        Bidirectional(LSTM(units=64, return_sequences=True), merge_mode='concat'),  
        Dropout(rate=0.2),  # 드롭아웃 20%
        # 시퀀스 데이터의 각 타임 스텝에 대해 Dense 레이어를 적용. features_dim은 출력 특성의 수(PCA 차원 수)
        TimeDistributed(Dense(features_dim)),  
        Activation(activation='tanh')])  # tanh 활성화 함수를 사용하여 출력. 최종 출력 형태는 (None, 10, 1)
    return Model(x, model(x))
print("생성기 레이어 생성 함수 정의 완료!")

생성기 레이어 생성 함수 정의 완료!


- Bidrectional
    - merge_mode : 두 방향의 출력을 어떻게 결합할지
        - sum : 순방향과 역방향 출력을 더함
        - mul : 곱함
        - ave : 평균
        - concat : 연결하여 결합. 차원 수 2배
- upSampling1D : 업샘플링하여 시퀀스의 길이를 늘림
    - size = 1 : 시퀀스의 길이는 변하지 않음
    - size = 2 : 시퀀스의 타임 스텝이 2배가 됨
- Dropout : 일부 뉴런을 학습에서 제외하여 과적합을 방지하는 역할
    - rate : 드롭아웃 비율
- TimeDistributed : 입력 시퀀스의 각 타임 스텝에 대해 지정된 레이어를 독립적으로 적용
- Activation : tanh
    - 신경망의 출력을 특정 범위로 변환하는 데 사용. 
    - 신경망의 각 뉴런이 출력하는 값을 변환해 비선형성을 도입. 모델이 더 복잡한 패턴을 학습하도록 함
    - tanh 활성화 함수는 출력을 -1과 1 사이로 조절(주성분 분석을 위한 출력 결과)

- MTadGAN의 판별자 critic_x 레이어를 만드는 함수를 정의한다. 여기에서 critic_x는 실제의 학습 데이터와 숨은 공간에서 생성된 가상의 데이터를 구별해내는 판별자 네트워크를 말한다. 여기에서 critic_x는 실제의 학습 데이터와 숨은 공간에서 생성된 가상의 데이터를 구별해내는 판별자 네트워크를 말한다. 즉, 실제의 데이터 X 에서 나오는 시계열 시퀀스들과 생성자 G(z) 에서 생성된 가짜의 시계열 시퀀스들을 분별해내는 함수이다.

In [14]:
# win_size에 따라 커널 사이즈 조정
if win_size >= 30:
    k_size = 5
else:
    k_size = 2

def build_critic_x_layer(input_shape):
    """critic_x 레이어 생성
    Args:
    critic_x_input_shape: [10, 1]
    Returns:
    critic_x 모델
    """
    x = Input(shape=input_shape)
    model = tf.keras.models.Sequential([
        Conv1D(filters=64, kernel_size=k_size),  # 64개 필터, 
        LeakyReLU(alpha=0.2),  # LeakyReLU 활성화 함수, 
        Dropout(rate=0.25),  # 드롭아웃 25%
        Conv1D(filters=64, kernel_size=k_size),  
        LeakyReLU(alpha=0.2),  
        Dropout(rate=0.25),  
        Conv1D(filters=64, kernel_size=k_size),  
        LeakyReLU(alpha=0.2),  
        Dropout(rate=0.25),  
        Conv1D(filters=64, kernel_size=k_size),  
        LeakyReLU(alpha=0.2),  
        Dropout(rate=0.25),  
        Flatten(),  # 1차원으로 평탄화
        Dense(units=1)  # 1개의 유닛을 가진 Dense 레이어
    ])
    return Model(x, model(x))

print("critic_x 레이어 생성 함수 정의 완료")

critic_x 레이어 생성 함수 정의 완료


- Conv1D : 
    - kernel_size : 합성곱 연산 수행 시의 사용하는 커널의 크기
    - 커널 사이즈 2 : 작은 구간에서 특징 추출. 세부적인 패턴 인식. 인접한 두 타임 스텝에서 특징 추출
    - 커널 사이즈 5 : 더 넓은 구간서 추출. 큰 패턴 트렌드 인식. 인접 다섯 타임 스텝에서 특징 추출
    - 커널이 클수록 많은 계산 필요로 함
- LeakyReLU : 활성화함수 중 하나
    - 음수 입력에 대해서도 작은 기울기를 갖게 하여 죽은 뉴런 문제를 해결
    - alpha : 음수입력에 대한 기울기. 보통 0.01 혹은 0.2
    - 너무 큰 alpha 값은 음수 입력에 대한 기울기를 크게 하여 모델의 성능 저하
    - 너무 작은 alpha는 ReLU와 다를 게 없음
    - ReLU는 alpha=0인 경우
- ReLU : ReLU 계열의 활성화함수의 기반이 되는 활성화함수
    - 간단하고 계산 비용이 적으며, 기울기 소실 문제를 해결하는 활성화함수
    - 학습 속도와 예측 속도가 빠르며, 깊은 신경망에서도 효과적
    - 죽은 뉴런 문제와 출력의 비대칭성

- MTadGAN의 판별자 critic_z 레이어를 만드는 함수를 정의한다. critic_z는 실제의 학습 데이터 샘플로부터 encoding 된 벡터와 그냥 숨은 공간에서 샘플된 벡터 사이를 구별해내는 역할을 한다. 구별해내는 판별자 네트워크를 말한다. 즉, 그림 19에서 보듯이 무작위로 택한 잠재공간 내의 표본 z 와 학습 데이터 샘플로부터 생성자 E(x) 를 써서 인코딩(encoding)된 표본을 분별하는 함수(네트워크)이다. 즉 그림 19에서 보듯이 critic_z는 잠재공간(embedding space) 내에서의 판별자이고 critic_x는 이와 반대로 실제 데이터 공간에서의 판별자가 된다.

In [15]:
def build_critic_z_layer(input_shape):
    """critic_z 레이어 생성
    Args:
    critic_z_input_shape: [20, 1]
    Returns:
    critic_z 모델
    """
    x = Input(shape=input_shape)
    model = tf.keras.models.Sequential([
        Flatten(),  # 입력을 1차원으로 평탄화
        Dense(units=100),  # 100개의 유닛을 가진 Dense 레이어
        LeakyReLU(alpha=0.2),  # LeakyReLU 활성화 함수, 알파=0.2
        Dropout(rate=0.2),  # 드롭아웃 20%
        Dense(units=100),  
        LeakyReLU(alpha=0.2), 
        Dropout(rate=0.2),  
        Dense(units=1)  # 1개의 유닛을 가진 Dense 레이어
    ])
    return Model(x, model(x))

print("critic_z 레이어 생성 함수 정의 완료")

critic_z 레이어 생성 함수 정의 완료


- MTadGAN의 Wasserstein 로스 함수를 정의한다. Wasserstein 로스 함수는 두 확률분포 사이의 거리를 주는 함수로서 판별자 네트워크의 학습에 쓰이는 로스함수이다.

In [16]:
def wasserstein_loss(y_true, y_pred):
    """Wasserstein 손실 계산
    Args:
    y_true: 실제 값
    y_pred: 예측 값
    Returns:
    loss: Wasserstein 손실
    """
    return K.mean(y_true * y_pred)
print("Wasserstein 손실 함수 정의 완료")

Wasserstein 손실 함수 정의 완료


- MTadGAN에서 필요로 하는 checkpoints 디렉터리, 기타 앞에서 정의한 네트워크 파라미터 값을 복사한다. checkpoints 디렉터리는 학습된 MTadGAN 모형을 저장하는 데에 쓰인다.

In [17]:
ckpt_dir = os.path.join(os.getcwd(), 'checkpoints')
os.makedirs(ckpt_dir, exist_ok=True)
network_dir = os.path.join(os.getcwd(), 'networks')
os.makedirs(network_dir, exist_ok=True)
# 네트워크 플랏
plot_network = params['plot_network']
# 학습 파라미터
batch_size = params['batch_size']
n_critics = params['n_critic']
epochs = params['epochs']
# 층 파라미터
shape = params['shape']
window_size = shape[0]
feat_dim = shape[1]
latent_dim = params['latent_dim']
encoder_input_shape = params['encoder_input_shape']
generator_input_shape = params['generator_input_shape']
critic_x_input_shape = params['critic_x_input_shape']
critic_z_input_shape = params['critic_z_input_shape']
encoder_reshape_shape = params['encoder_reshape_shape']
generator_reshape_shape = params['generator_reshape_shape']
print('MTadGAN 초기화')
print("latent_dim= ", latent_dim)
print("shape= ", shape)
print("encoder_input_shape= ", encoder_input_shape)
print("generator_input_shape= ", generator_input_shape)
print("critic_x_input_shape= ", critic_x_input_shape)
print("critic_z_input_shape= ", critic_z_input_shape)
print("encoder_reshape_shape= ", encoder_reshape_shape)
print("generator_reshape_shape= ", generator_reshape_shape)

MTadGAN 초기화
latent_dim=  20
shape=  [10, 3]
encoder_input_shape=  [10, 3]
generator_input_shape=  [20, 1]
critic_x_input_shape=  [10, 3]
critic_z_input_shape=  [20, 1]
encoder_reshape_shape=  [20, 1]
generator_reshape_shape=  [10, 1]


- MTadGAN의 encoder, generator 및 판별자(critic_x, critic_z) 네트워크의 인스턴스를 정의한다. 동시에 학습 계산을 위한 최적화 알고리즘으로 Adam 함수를 도입한다. 학습률의 값은 learning_rate = 0.0005로 정의한다.

In [18]:
learning_rate =0.0005
# 각 함수에 입력 층 부여
encoder = build_encoder_layer(input_shape=encoder_input_shape,
                              encoder_reshape_shape=encoder_reshape_shape)
generator = build_generator_layer(input_shape=generator_input_shape,
                                  generator_reshape_shape=generator_reshape_shape)
critic_x = build_critic_x_layer(input_shape=critic_x_input_shape)
critic_z = build_critic_z_layer(input_shape=critic_z_input_shape)
optimizer = tf.keras.optimizers.Adam(learning_rate)
print("encoder generator critic_x critic_z optimizer 인스턴스 정의 완료")

encoder generator critic_x critic_z optimizer 인스턴스 정의 완료


- 판별자 네트워크와 encoder-generator 네트워크의 입출력 구조를 명시한다.

In [19]:
z = Input(shape=(latent_dim, 1))
x = Input(shape=shape)
x_ = generator(z)
z_ = encoder(x)
fake_x = critic_x(x_)
valid_x = critic_x(x)
interpolated_x = RandomWeightedAverage(batch_size)([x, x_])
critic_x_model = Model(inputs=[x, z], outputs=[valid_x, fake_x, interpolated_x])
fake_z = critic_z(z_)
valid_z = critic_z(z)
interpolated_z = RandomWeightedAverage(batch_size)([z, z_])
critic_z_model = Model(inputs=[x, z], outputs=[valid_z, fake_z, interpolated_z])
z_gen = Input(shape=(latent_dim, 1))
x_gen_ = generator(z_gen)
x_gen = Input(shape=shape)
z_gen_ = encoder(x_gen)
x_gen_rec = generator(z_gen_)
fake_gen_x = critic_x(x_gen_)
fake_gen_z = critic_z(z_gen_)
encoder_generator_model = Model([x_gen, z_gen], [fake_gen_x, fake_gen_z, x_gen_rec])

- z는 잠재 공간(latent space)에서 입력을 받는 텐서. 입력 형태는 (latent_dim, 1)
- x는 실제 데이터 입력을 받는 텐서입니다. 입력 형태는 shape로 정의
- x_는 생성기(generator)를 통해 잠재 공간 z로부터 생성된 가짜 데이터
- z_는 실제 데이터 x를 인코더(encoder)를 통해 변환한 잠재 표현
- fake_x는 critic_x 모델이 생성된 가짜 데이터 x_에 대해 평가한 결과
- valid_x는 critic_x 모델이 실제 데이터 x에 대해 평가한 결과
- interpolated_x는 실제 데이터 x와 생성된 데이터 x_의 랜덤 가중 평균. WGAN-GP(Wasserstein GAN with Gradient Penalty)에서 사용
- critic_x_model은 critic_x의 모델로, 실제 데이터 x, 잠재 공간 z를 입력으로 받아 valid_x, fake_x, interpolated_x를 출력으로 함
- fake_z는 critic_z 모델이 인코더로부터 얻은 잠재 표현 z_에 대해 평가한 결과
-  valid_z는 critic_z 모델이 잠재 공간 z에 대해 평가한 결과
- z_gen은 생성기에서 입력을 받는 잠재 공간 z를 나타냄. 입력 형태는 (latent_dim, 1)
- x_gen_는 생성기를 통해 잠재 공간 z_gen에서 생성된 가짜 데이터
- x_gen은 실제 데이터의 입력을 받는 텐서. 입력 형태는 shape로 정의
- z_gen_는 실제 데이터 x_gen을 인코더를 통해 변환한 잠재 표현
- x_gen_rec는 인코더를 통해 얻은 잠재 표현 z_gen_를 다시 생성기를 통해 복원한 데이터. 재구성된 데이터
- fake_gen_x는 critic_x 모델이 생성된 가짜 데이터 x_gen_에 대해 평가한 결과
- fake_gen_z는 critic_z 모델이 인코더를 통해 얻은 잠재 표현 z_gen_에 대해 평가한 결과
- ncoder_generator_model은 인코더와 생성기를 결합한 모델로, 실제 데이터 x_gen과 잠재 공간 z_gen을 입력으로 받아 가짜 데이터에 대한 평가 결과 fake_gen_x, 잠재 표현에 대한 평가 결과 fake_gen_z, 그리고 재구성된 데이터 x_gen_rec을 출력으로 함

- 이미 critic_x 모형, critic_z 모형 그리고 encoder-generator 모형의 세 가지가 학습되어 저장되어 있으면 이를 불러오고, 네트워크 구조를 그래픽 파일로 저장하며 입출력 구조를 출력한다.

In [20]:
if os.path.isfile(os.path.join(ckpt_dir, 'critic_x_model.h5')):
    print("load critic_x weights")
    critic_x_model.load_weights(os.path.join(ckpt_dir, 'critic_x_model.h5'))
if os.path.isfile(os.path.join(ckpt_dir, 'critic_z_model.h5')):
    print("load critic_z weights")
    critic_z_model.load_weights(os.path.join(ckpt_dir, 'critic_z_model.h5'))
if os.path.isfile(os.path.join(ckpt_dir, 'encoder_generator_model.h5')):
    print("load encoder_generator weights")
    encoder_generator_model.load_weights(os.path.join(
                                            ckpt_dir,
                                            'encoder_generator_model.h5'))
critic_x_model.summary()
critic_z_model.summary()
encoder_generator_model.summary()

if plot_network:
    plot_model(critic_x_model,
            to_file=os.path.join(network_dir, 'critic_x_model_tf2.png'),
            show_shapes=True,
            expand_nested=True)
    plot_model(critic_z_model,
        to_file=os.path.join(network_dir, 'critic_z_model_tf2.png'),
        show_shapes=True,
        expand_nested=True)
    plot_model(encoder_generator_model,
        to_file=os.path.join(network_dir,'enc_gen_model_tf2.png'),
        show_shapes=True,
        expand_nested=True)
print("critic_x_model critic_z_model encode_generator_model 정의 완료")

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 20, 1)]      0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 10, 3)]      0                                            
__________________________________________________________________________________________________
model_1 (Functional)            (None, 10, 3)        133205      input_5[0][0]                    
__________________________________________________________________________________________________
model_2 (Functional)            (None, 1)            25601       model_1[0][0]                    
                                                                 input_6[0][0]              

- critic_x 네트워크를 학습하는 과정을 수행하는 함수를 정의한다. 로스함수를 정의하고 gradient 계산함수, 그리고 Adam 최적화 함수를 포함한다.

In [21]:
@tf.function
def critic_x_train_on_batch(x, z, valid, fake, delta):
    # GradientTape를 사용하여 그래디언트를 추적
    with tf.GradientTape() as tape:
        # critic_x_model을 통해 valid_x, fake_x, interpolated 값을 계산 (training=True 설정)
        (valid_x, fake_x, interpolated) = critic_x_model(inputs=[x, z], training=True)
        
        # GradientTape를 사용하여 interpolated의 그래디언트를 추적
        with tf.GradientTape() as gp_tape:
            gp_tape.watch(interpolated)
            pred = critic_x(interpolated, training=True)
        
        # interpolated에 대한 그래디언트를 계산
        grads = gp_tape.gradient(pred, interpolated)[0]
        grads = tf.square(grads)
        ddx = tf.sqrt(1e-8 + tf.reduce_sum(grads, axis=np.arange(1, len(grads.shape))))
        
        # Gradient Penalty 손실 계산
        gp_loss = tf.reduce_mean((ddx - 1.0) ** 2)
        
        # Wasserstein 손실 계산
        loss = tf.reduce_mean(wasserstein_loss(valid, valid_x))
        loss += tf.reduce_mean(wasserstein_loss(fake, fake_x))
        
        # 최종 손실에 Gradient Penalty 손실을 추가
        loss += gp_loss * 10.0
    
    # 손실에 대한 critic_x_model의 가중치 그래디언트를 계산
    gradients = tape.gradient(loss, critic_x_model.trainable_weights)
    
    # 옵티마이저를 사용하여 가중치 업데이트
    optimizer.apply_gradients(zip(gradients, critic_x_model.trainable_weights))
    
    return loss

print("critic_x_train_on_batch 정의 완료!")

critic_x_train_on_batch 정의 완료!


- critic_z 네트워크를 학습하는 과정을 수행하는 함수를 정의한다. 로스함수를 정의하고 gradient 계산함수, 그리고 Adam 최적화 함수를 포함한다.

In [22]:
@tf.function
def critic_z_train_on_batch(x, z, valid, fake, delta):
    # GradientTape를 사용하여 그래디언트를 추적
    with tf.GradientTape() as tape:
        # critic_z_model을 통해 valid_z, fake_z, interpolated 값을 계산 (training=True 설정)
        (valid_z, fake_z, interpolated) = critic_z_model(inputs=[x, z], training=True)
        
        # GradientTape를 사용하여 interpolated의 그래디언트를 추적
        with tf.GradientTape() as gp_tape:
            gp_tape.watch(interpolated)
            pred = critic_z(interpolated, training=True)
        
        # interpolated에 대한 그래디언트를 계산
        grads = gp_tape.gradient(pred, interpolated)[0]
        grads = tf.square(grads)
        ddx = tf.sqrt(1e-8 + tf.reduce_sum(grads, axis=np.arange(1, len(grads.shape))))
        
        # Gradient Penalty 손실 계산
        gp_loss = tf.reduce_mean((ddx - 1.0) ** 2)
        
        # Wasserstein 손실 계산
        loss = tf.reduce_mean(wasserstein_loss(valid, valid_z))
        loss += tf.reduce_mean(wasserstein_loss(fake, fake_z))
        
        # 최종 손실에 Gradient Penalty 손실을 추가
        loss += gp_loss * 10.0
    
    # 손실에 대한 critic_z_model의 가중치 그래디언트를 계산
    gradients = tape.gradient(loss, critic_z_model.trainable_weights)
    
    # 옵티마이저를 사용하여 가중치 업데이트
    optimizer.apply_gradients(zip(gradients, critic_z_model.trainable_weights))
    
    return loss

print("critic_z_train_on_batch 정의 완료")

critic_z_train_on_batch 정의 완료


- encoder-generator 네트워크를 학습하는 과정을 수행하는 함수를 정의한다. 로스함수를 정의하고 gradient 함수, 그리고 Adam 최적화 함수를 포함한다.

In [23]:
@tf.function
def enc_gen_train_on_batch(x, z, valid):
    # GradientTape를 사용하여 그래디언트를 추적
    with tf.GradientTape() as tape:
        # encoder_generator_model을 통해 fake_gen_x, fake_gen_z, x_gen_rec 값을 계산 (training=True 설정)
        (fake_gen_x, fake_gen_z, x_gen_rec) = encoder_generator_model(inputs=[x, z], training=True)
        
        # 실제 데이터 x와 재구성된 데이터 x_gen_rec을 차원 축소
        x = tf.squeeze(x)
        x_gen_rec = tf.squeeze(x_gen_rec)
        
        # Wasserstein 손실 계산
        loss = tf.reduce_mean(wasserstein_loss(valid, fake_gen_x))
        loss += tf.reduce_mean(wasserstein_loss(valid, fake_gen_z))
        
        # 재구성 손실(MSE) 계산 및 추가
        loss += tf.keras.losses.MSE(x, x_gen_rec) * 10
        
        # 손실 평균 계산
        loss = tf.reduce_mean(loss)
    
    # 손실에 대한 encoder_generator_model의 가중치 그래디언트를 계산
    gradients = tape.gradient(loss, encoder_generator_model.trainable_weights)
    
    # 옵티마이저를 사용하여 가중치 업데이트
    optimizer.apply_gradients(zip(gradients, encoder_generator_model.trainable_weights))
    
    return loss

print("enc_gen_train_on_batch 정의 완료")

enc_gen_train_on_batch 정의 완료


- 학습률 Learning rate = 0.0005, 반복 횟수 epochs=30으로 학습을 진행하며 각 epoch 끝날 때마다 로스 함수들을 출력한다. 학습 부분은 로스 함수의 최적화 과정에서 초기 네트워크 상태를 무작위 상태로부터 시작하거나 또는 기존의 네트워크의 모형을 불러와서 시작하거나 하므로 그 시작 상태에 따라 출력 결과가 실행 시마다 조금씩 달라진다. 즉, 최종적으로 학습된 네트워크 모형이 달라질 수 있는 것이다. 따라서 이렇게 조금씩 달라진 네트워크 모형을 써서 수행하는 테스트 결과 또한 실행 시마다 조금씩 그 결과가 달라진다.

In [27]:
# 학습
X = X.reshape((-1, shape[0], feat_dim))  # feat_dim = 주성분 차원
X_ = np.copy(X)
fake = np.ones((batch_size, 1), dtype=np.float32)  # 가짜 레이블 (1)
valid = -np.ones((batch_size, 1), dtype=np.float32)  # 진짜 레이블 (-1)
delta = np.ones((batch_size, 1), dtype=np.float32)  # 델타 (1)
for epoch in range(1, epochs + 1):
    np.random.shuffle(X_)  # 데이터를 섞음
    epoch_g_loss = []  # 생성기 손실을 저장할 리스트
    epoch_cx_loss = []  # critic_x 손실을 저장할 리스트
    epoch_cz_loss = []  # critic_z 손실을 저장할 리스트
    minibatches_size = batch_size * n_critics
    num_minibatches = int(X_.shape[0] // minibatches_size)
    for i in range(num_minibatches):
        minibatch = X_[i * minibatches_size:(i + 1) * minibatches_size]
        # 크리틱 학습
        critic_x.trainable = True
        critic_z.trainable = True
        generator.trainable = False
        encoder.trainable = False
        for j in range(n_critics):
            x = minibatch[j * batch_size:(j + 1) * batch_size]
            z = np.random.normal(size=(batch_size, latent_dim, 1))
            epoch_cx_loss.append(critic_x_train_on_batch(x, z, valid, fake, delta))
            epoch_cz_loss.append(critic_z_train_on_batch(x, z, valid, fake, delta))
        critic_x.trainable = False
        critic_z.trainable = False 
        generator.trainable = True
        encoder.trainable = True 
        # 인코더와 생성기 학습
        epoch_g_loss.append(enc_gen_train_on_batch(x, z, valid))
    cx_loss = np.mean(np.array(epoch_cx_loss), axis=0)
    cz_loss = np.mean(np.array(epoch_cz_loss), axis=0)
    g_loss = np.mean(np.array(epoch_g_loss), axis=0)
    print('Epoch: {}/{}, [Dx loss: {}] [Dz loss: {}] [G loss: {}]'.format(epoch, epochs, cx_loss, cz_loss, g_loss)) 
critic_x_model.save_weights(os.path.join(ckpt_dir, 'critic_x_model.h5'), save_format='h5')
critic_z_model.save_weights(os.path.join(ckpt_dir, 'critic_z_model.h5'), save_format='h5')
encoder_generator_model.save_weights(os.path.join(ckpt_dir, 'encoder_generator_model.h5'), save_format='h5')

Epoch: 1/30, [Dx loss: -0.5098100304603577] [Dz loss: -8.854000091552734] [G loss: 12.369721412658691]
Epoch: 2/30, [Dx loss: 0.32244664430618286] [Dz loss: -7.961201190948486] [G loss: 17.240400314331055]
Epoch: 3/30, [Dx loss: 0.3969837725162506] [Dz loss: -6.578747749328613] [G loss: 16.552921295166016]
Epoch: 4/30, [Dx loss: 0.17430128157138824] [Dz loss: -6.177322864532471] [G loss: 11.285940170288086]
Epoch: 5/30, [Dx loss: 0.024358348920941353] [Dz loss: -6.293560981750488] [G loss: 13.72192096710205]
Epoch: 6/30, [Dx loss: -0.5393539667129517] [Dz loss: -7.561406135559082] [G loss: 14.157665252685547]
Epoch: 7/30, [Dx loss: 0.014545773155987263] [Dz loss: -9.815912246704102] [G loss: 11.349740982055664]
Epoch: 8/30, [Dx loss: 0.14468812942504883] [Dz loss: -9.328973770141602] [G loss: 8.92959213256836]
Epoch: 9/30, [Dx loss: -0.7452656030654907] [Dz loss: -8.527708053588867] [G loss: 0.8259146809577942]
Epoch: 10/30, [Dx loss: -0.29398059844970703] [Dz loss: -7.268407344818115]

ImportError: `save_weights` requires h5py when saving in hdf5.

## [단계 ⑥] 테스트 데이터에 대한 계산 수행
⑥-1. 테스트 데이터에 대한 계산 수행
- 예측 함수 predict()를 정의한다. 이 함수는 입력 데이터에 encoder-generator 네트워크를 작용하여 재현된 데이터 y_hat을 출력하고 또한 critic_x 판별자 값을 출력한다.

In [35]:
def predict(X):
    """초기 설정된 객체를 사용하여 값을 예측.
    Args:
    X (ndarray): 모델을 위한 입력 시퀀스를 포함하는 N차원 배열.
    Returns:
    ndarray:
    각 입력 시퀀스에 대한 재구성을 포함하는 N차원 배열.
    ndarray:
    각 입력 시퀀스에 대한 비평 점수를 포함하는 N차원 배열.
    """
    X = X.reshape((-1, shape[0], feat_dim)) # feat_dim : 특징 차원 
    z_ = encoder.predict(X)
    y_hat = generator.predict(z_)
    critic = critic_x.predict(X)
    return y_hat, critic
print("predict() 정의: 완료")

predict() 정의: 완료


- 미리 준비된 테스트 데이터 파일 “./data/preprocessed/test/Test07_NG_dchg.csv”와 anomaly 파일인 “./data/preprocessed/test/Test07_NG_dchg_Label.csv”를 지정한다.

In [36]:
# 테스트 데이터를 이용한 예측
# 테스트 데이터 불러오기
args=arguments(signal_file='../../../../data/preprocessed/test/Test07_NG_dchg.csv',
    timest_form=0,
    anomaly_file='../../../../data/preprocessed/test/Test07_NG_dchg_Label.csv',
    mode='predict',
    aggregate_interval=1,
    regate_interval=1)
print("argments for prediction mode")
print("----------------------")
print("args.timest_form :", args.timest_form)
print("args.signal_file :", args.signal_file)
print("args.anomaly_file :", args.anomaly_file)
print("args.mode :", args.mode)
print("args.aggregate_interval :", args.aggregate_interval)
print("----------------------\n")

argments for prediction mode
----------------------
args.timest_form : 0
args.signal_file : ../../../../data/preprocessed/test/Test07_NG_dchg.csv
args.anomaly_file : ../../../../data/preprocessed/test/Test07_NG_dchg_Label.csv
args.mode : predict
args.aggregate_interval : 1
----------------------



In [37]:
file_path = args.signal_file 
df_test1 = pd.read_csv(file_path) # 테스트 데이터 데이터 프레임 형태로 읽기

In [38]:
######## PCA를 이용한 차원 축소 ########
data_0 = df_test1 # 복제
# (lags_n=0, diffs_n=0, smooth_n=0)
data_1 = diff_smooth_df(data_0, lags_n, diffs_n, smooth_n)
pca = PCA(n_components=features_dim)
data = pca.fit_transform(data_1) # PCA를 이용한 차원 축소
df_1 = []
for i in range(len(data)):
    row=[i+1] # 정수 인덱스 부여
    for jj in range(features_dim):
        row.append(data[i][jj])
    df_1.append(row)
df = pd.DataFrame(df_1)
columns_new = ['date']
for i in range(1, features_dim+1):
    pca_i = 'pca_%s' % str(i)
    columns_new.append(pca_i)
df.columns=columns_new
print("After PCA reduction of test data df.shape = ", df.shape)
print("----------------------")
print(df.head(5))
print("----------------------")
print("\n")

After PCA reduction of test data df.shape =  (4594, 4)
----------------------
   date     pca_1     pca_2     pca_3
0     1 -0.412384  0.583294  0.027636
1     2 -0.406828  0.567897  0.026531
2     3 -0.368964  0.575908  0.015944
3     4 -0.430935  0.547518  0.026097
4     5 -0.425375  0.532121  0.025034
----------------------




- MTadGAN 모형에 입력하기 위하여 테스트 데이터에 대한 집합화(aggregate)를 수행한다. time_segments_aggregate() 함수에 대해서는 위의 코드를 참조한다.

In [39]:
# 시간 집합화 수행
#
X, index = time_segments_aggregate(df, interval=args.aggregate_interval, time_column='date')
print("Test signal data (after time_segments_aggregate) = ", X.shape)
print("----------------------")
print(X[:5])
print("----------------------")
print("\n")

Test signal data (after time_segments_aggregate) =  (4594, 3)
----------------------
[[-0.41238433  0.58329384  0.02763632]
 [-0.40682767  0.56789719  0.02653118]
 [-0.36896407  0.57590827  0.01594388]
 [-0.43093477  0.54751791  0.02609742]
 [-0.42537477  0.532121    0.02503421]]
----------------------




- 스케일링에 의한 정규화 과정을 수행한다. 테스트 데이터를 [-1,+1] 구간 안의 값으로 정규화한다. 

In [40]:
# 최대 - 최소 정규화 적용
X = SimpleImputer().fit_transform(X)
X = MinMaxScaler(feature_range=(-1, 1)).fit_transform(X)
print("Test X (after MinMaxScaler) = ", X.shape)
print("----------------------")
print(X[:5])
print("----------------------")
print("\n")

Test X (after MinMaxScaler) =  (4594, 3)
----------------------
[[-0.98179729  1.          0.02048555]
 [-0.97807573  0.96581515  0.01806001]
 [-0.95271668  0.98360199 -0.0051768 ]
 [-0.9942214   0.9205675   0.01710801]
 [-0.99049761  0.88638206  0.01477449]]
----------------------




- MTadGAN의 구조에 맞게 입력할 수 있도록 테스트 데이터에 rolling_window_sequences() 함수를 적용하여 이웃하는 데이터 값으로 윈도우 길이(win_size=10)만큼씩 묶어서 저장한다.

In [41]:
# rolling window sequences 처리
X, y, X_index, y_index = rolling_window_sequences(X, index, window_size= win_size,
 target_size=1, step_size=1, target_column=0 ) 
print("X shape (after rolling_window_seq): {}".format(X.shape))
print("X : ")
print(X[:3, :5])
print("X index shape: {}".format(X_index.shape))
print("y shape: {}".format(y.shape))
print("y : ")
print(y[:5])
print("y index shape: {}".format(y_index.shape))
print("\n")

X shape (after rolling_window_seq): (4584, 10, 3)
X : 
[[[-9.81797293e-01  1.00000000e+00  2.04855463e-02]
  [-9.78075731e-01  9.65815151e-01  1.80600130e-02]
  [-9.52716680e-01  9.83601990e-01 -5.17680324e-03]
  [-9.94221404e-01  9.20567495e-01  1.71080075e-02]
  [-9.90497609e-01  8.86382057e-01  1.47744897e-02]]

 [[-9.78075731e-01  9.65815151e-01  1.80600130e-02]
  [-9.52716680e-01  9.83601990e-01 -5.17680324e-03]
  [-9.94221404e-01  9.20567495e-01  1.71080075e-02]
  [-9.90497609e-01  8.86382057e-01  1.47744897e-02]
  [-9.60544141e-01  9.71167281e-01 -9.41994759e-04]]

 [[-9.52716680e-01  9.83601990e-01 -5.17680324e-03]
  [-9.94221404e-01  9.20567495e-01  1.71080075e-02]
  [-9.90497609e-01  8.86382057e-01  1.47744897e-02]
  [-9.60544141e-01  9.71167281e-01 -9.41994759e-04]
  [-9.79474891e-01  9.25992926e-01  1.28564571e-02]]]
X index shape: (4584,)
y shape: (4584, 1)
y : 
[[-0.97946451]
 [-0.99049122]
 [-0.99048524]
 [-0.97807193]
 [-0.97945812]]
y index shape: (4584,)




- MTadGAN에서의 Anomaly 클래스 및 이상탐지 관련 속성 함수들을 정의한다. 여기서는 Anomaly 클래스에서 주로 쓰이는 주요한 속성함수에 대하여 간략하게 설명한다. 각 함수의 입력 변수는 생략한다.  
    - _find_threshold() : 에러 값들로부터 최적의 threshold 값을 찾는다.
    - _fixed_threshold() : 입력 k 변수의 값에 따라 에러의 threshold를 구한다.
    - _compute_scores() : Anomaly score를 계산한다.
    - _find_window_sequences() : 연속된 이상 값들의 sequence를 찾는다.
    - find_anomalies() : 이상 값을 가지는 연속된 에러값의 sequence를 찾는다.
    - _compute_critic_score() : 판별자 score에서 얻는 이상 값의 배열을 계산한다. 
    - _reconstruction_errors() : 데이터 재현 에러 값을 계산한다. 이는 입력 데이터와 y_hat 사이의차이값으로부터 구해진다.
    - score_anomalies() : 판별자 score와 재현 에러의 결합으로 최종 이상 score를 얻는다.


In [ ]:
"""
시계열 이상 탐지 함수.
일부 구현은 논문 https://arxiv.org/pdf/1802.04431.pdf을 참고하였습니다.
"""
class Anomaly(object):
    def __init__(self):
        pass

    def _deltas(self, errors, epsilon, mean, std):
        """평균 및 표준편차 델타를 계산.
        delta_mean = mean(errors) - epsilon(임계값) 이하의 모든 오류들의 평균
        delta_std = std(errors) - epsilon(임계값) 이하의 모든 오류들의 표준편차
        Args:
        errors (ndarray): 오류 배열.
        epsilon (ndarray): 임계값.
        mean (float): 오류들의 평균.
        std (float): 오류들의 표준편차.
        Returns:
        float, float:
        * delta_mean.
        * delta_std.
        """
        below = errors[errors <= epsilon]
        if not len(below):
            return 0, 0
        return mean - below.mean(), std - below.std()
    # anomalies.py (2)
    def _count_above(self, errors, epsilon):
        """epsilon 이상인 오류와 연속된 시퀀스의 수를 계산
        연속된 시퀀스는 시프트하고 원래 값이 true였던 위치의 변화를
        계산하여 그 위치에서 시퀀스가 시작되었음을 의미
        Args:
        errors (ndarray): 오류 배열.
        epsilon (ndarray): 임계값.
        Returns:
        int, int:
        * epsilon 이상인 오류의 수.
        * epsilon 이상인 연속된 시퀀스의 수.
        """
        # errors 배열에서 epsilon보다 큰 값인지 여부 배열
        above = errors > epsilon
        # epsilon보다 큰 오류의 총 수를 계산
        total_above = len(errors[above])
        # above 배열을 pandas Series로 변환
        above = pd.Series(above)
        # above Series를 1만큼 시프트(레코드를 한 칸씩 밈)
        shift = above.shift(1)
        # above와 shift된 값 간의 변화를 계산(서로 다른 경우가 true)
        change = above != shift
        # epsilon보다 큰 연속된 시퀀스의 수를 계산
        total_consecutive = sum(above & change)
        # 결과를 반환
        return total_above, total_consecutive
    # anomalies.py (3)
    def _z_cost(self, z, errors, mean, std):
        """z 값이 얼마나 나쁜지를 계산
        원래 공식::
        (delta_mean/mean) + (delta_std/std)
        ------------------------------------------------------
        number of errors above + (number of sequences above)^2
        이는 `z`의 "좋음"을 계산하며, 값이 높을수록 `z`가 더 좋다는 것을 의미
        이 경우, 이 값을 반전(음수로 만듦)하여 비용 함수로 변환
        나중에 scipy.fmin을 사용하여 이를 최소화
        
        Args:
        z (ndarray): 비용 점수가 계산될 값.
        errors (ndarray): 오류 배열.
        mean (float): 오류들의 평균.
        std (float): 오류들의 표준편차.
        
        Returns float: z의 비용.
        """
        
        # epsilon 값을 평균 + z * 표준편차로 계산
        epsilon = mean + z * std
        # epsilon을 사용하여 평균 및 표준편차 델타를 계산
        delta_mean, delta_std = self._deltas(errors, epsilon, mean, std)
        # epsilon보다 큰 오류와 연속된 시퀀스의 수를 계산
        above, consecutive = self._count_above(errors, epsilon)
        # 분자(numerator)를 계산합니다. (반전된 값)
        numerator = -(delta_mean / mean + delta_std / std)
        # 분모(denominator)를 계산합니다.
        denominator = above + consecutive ** 2
        # 분모가 0이면 무한대를 반환
        if denominator == 0:
            return np.inf
        
        # 최종 비용을 계산하여 반환
        return numerator / denominator
    # anomalies.py (4)
    def _find_threshold(self, errors, z_range):
        """이상적인 임계값 찾는 함수.
        이상적인 임계값은 z_cost 함수를 최소화하는 값.
        Scipy.fmin을 사용하여 z_range의 값들을 시작점으로 최소값을 탐색.

        Args:
        errors (ndarray): 오류 배열.
        z_range (list): scipy.fmin 함수의 시작점을 선택할 범위를 나타내는 두 값의 리스트.

        Returns: float: 계산된 임계값.
        """
        # 오류들의 평균을 계산.
        mean = errors.mean()
        
        # 오류들의 표준편차를 계산.
        std = errors.std()
        
        # z_range에서 최소값과 최대값을 가져옴.
        min_z, max_z = z_range
        
        # 최적의 z값을 저장할 변수 초기화.
        best_z = min_z
        
        # 최적의 비용을 무한대로 초기화.
        best_cost = np.inf

        # min_z부터 max_z까지 반복.
        for z in range(min_z, max_z):
            # fmin 함수를 사용하여 z에서 시작하는 최소 비용을 찾음.
            best = fmin(self._z_cost, z, args=(errors, mean, std), full_output=True, disp=False)
            
            # 최적의 z값과 비용을 가져옴.
            z, cost = best[0:2]
            
            # 현재 비용이 최적의 비용보다 작으면 갱신.
            if cost < best_cost:
                best_z = z[0]

        # 최적의 임계값을 계산하여 반환.
        return mean + best_z * std

    def _fixed_threshold(self, errors, k=3.0):
        """임계값 계산.
        고정된 임계값은 평균에서 k 표준편차만큼 떨어진 값으로 정의됨.

        Args:
        errors (ndarray): 오류 배열.

        Returns:
        float: 계산된 임계값.
        """
        # 오류들의 평균을 계산.
        mean = errors.mean()
        
        # 오류들의 표준편차를 계산.
        std = errors.std()

        # 고정된 임계값을 계산하여 반환.
        return mean + k * std
    
    # anomalies.py (5-1)
    def _find_sequences(self, errors, epsilon, anomaly_padding):
        """epsilon 이상인 값들의 시퀀스 탐색.
        다음 단계들을 따름:
        * epsilon 이상인 값을 표시하는 불리언 마스크 생성.
        * True 값 주위의 일정 범위의 오류를 True로 표시.
        * 이 마스크를 한 칸씩 시프트하고, 빈 공간은 False로 채움.
        * 시프트된 마스크와 원래 마스크를 비교하여 변화가 있는지 확인.
        * True였던 값이 변경된 지점을 시퀀스 시작으로 간주.
        * False였던 값이 변경된 지점을 시퀀스 종료로 간주.

        Args:
        errors (ndarray): 오류 배열.
        epsilon (float): 임계값. epsilon 이상의 모든 오류는 이상으로 간주.
        anomaly_padding (int): 발견된 이상 주위에 추가할 오류의 개수.

        Returns:
        ndarray, float:
        * 발견된 이상 시퀀스의 시작과 끝을 포함하는 배열.
        * 이상으로 간주되지 않은 최대 오류 값.
        """
        # epsilon 이상인 값을 표시하는 불린 마스크 생성.
        above = pd.Series(errors > epsilon)

        # True인 값의 인덱스를 찾음.
        index_above = np.argwhere(above.values)

        # True 값 주위의 일정 범위의 오류를 True로 표시.
        for idx in index_above.flatten():
            above[max(0, idx - anomaly_padding):min(idx + anomaly_padding + 1, len(above))] = True

        # 이 마스크를 한 칸씩 시프트하고, 빈 공간은 False로 채움.
        shift = above.shift(1).fillna(False)

        # 시프트된 마스크와 원래 마스크를 비교하여 변화가 있는지 확인.
        change = above != shift

        # 모든 값이 True이면 max_below는 0으로 설정.
        if above.all():
            max_below = 0
        else:
            # 이상으로 간주되지 않은 최대 오류 값을 찾음.
            max_below = max(errors[~above])

        # 시퀀스의 시작과 끝 인덱스를 찾음.
        index = above.index
        starts = index[above & change].tolist()
        ends = (index[~above & change] - 1).tolist()

        # 마지막 시퀀스가 종료되지 않았을 경우 종료 인덱스를 추가.
        if len(ends) == len(starts) - 1:
            ends.append(len(above) - 1)

        # 결과를 배열로 반환.
        return np.array([starts, ends]).T, max_below

    # anomalies.py (5-2)
    def _get_max_errors(self, errors, sequences, max_below):
        """각 이상 시퀀스의 최대 오류를 가져옴.
        또한 이상으로 간주되지 않은 최대 오류 값을 포함하는 행을 추가.
        각 시퀀스의 최대 오류를 포함하는 ``max_error`` 열과 시작 및 종료 인덱스를
        포함하는 ``start`` 및 ``stop`` 열이 있는 테이블을 내림차순으로 정렬하여 반환.

        Args:
        errors (ndarray): 오류 배열.
        sequences (ndarray): 이상 시퀀스의 시작과 끝을 포함하는 배열.
        max_below (float): 이상으로 간주되지 않은 최대 오류 값.

        Returns:
        pandas.DataFrame: ``start``, ``stop``, ``max_error`` 열을 포함하는 DataFrame 객체.
        """
        # 이상으로 간주되지 않은 최대 오류 값을 포함하는 초기 값 설정.
        max_errors = [{
            'max_error': max_below,
            'start': -1,
            'stop': -1
        }]

        # 각 시퀀스에 대해 최대 오류 값을 계산하고 추가.
        for sequence in sequences:
            start, stop = sequence
            sequence_errors = errors[start: stop + 1]
            max_errors.append({
                'start': start,
                'stop': stop,
                'max_error': max(sequence_errors)
            })

        # DataFrame으로 변환하고 최대 오류 값으로 내림차순 정렬.
        max_errors = pd.DataFrame(max_errors).sort_values('max_error', ascending=False)

        # 인덱스를 재설정하여 반환.
        return max_errors.reset_index(drop=True)
    
    # anomalies.py (6)
    def _prune_anomalies(self, max_errors, min_percent):
        """거짓 양성을 줄이기 위해 이상을 가지치기.
        다음 단계들을 따름:
        * 오류를 1단계 음수 방향으로 시프트하여 각 값을 다음 값과 비교.
        * 비교하지 않기 위해 마지막 행을 제거.
        * 각 행에 대한 백분율 증가를 계산.
        * ``min_percent`` 이하인 행을 찾음.
        * 이러한 행 중 가장 최근의 행의 인덱스를 찾음.
        * 해당 인덱스 위의 모든 시퀀스 값을 가져옴.

        Args:
        max_errors (pandas.DataFrame): ``start``, ``stop``, ``max_error`` 열을 포함하는 DataFrame 객체.
        min_percent (float):
        이상 간의 분리를 위한 최소 백분율. 창 시퀀스에서 가장 높은 비이상 오류와의 비교.

        Returns:
        ndarray: 가지치기된 이상들의 시작, 끝, max_error를 포함하는 배열.
        """
        # 오류를 1단계 음수 방향으로 시프트하여 다음 값과 비교.
        next_error = max_errors['max_error'].shift(-1).iloc[:-1]

        # 현재 오류 값을 가져옴.
        max_error = max_errors['max_error'].iloc[:-1]

        # 각 행에 대한 백분율 증가를 계산.
        increase = (max_error - next_error) / max_error

        # min_percent 이하인 행을 찾음.
        too_small = increase < min_percent

        # 모든 행이 min_percent 이하인 경우.
        if too_small.all():
            last_index = -1
        else:
            # min_percent 이상인 가장 최근의 행의 인덱스를 찾음.
            last_index = max_error[~too_small].index[-1]

        # 가지치기된 이상들의 시작, 끝, max_error를 포함하는 배열을 반환.
        return max_errors[['start', 'stop', 'max_error']].iloc[0: last_index + 1].values
    
    # anomalies.py (7)
    def _compute_scores(self, pruned_anomalies, errors, threshold, window_start):
        """이상의 점수를 계산.
        시퀀스에서 최대 오류에 비례하는 점수를 계산하고, 인덱스를 절대값으로 만들기 위해 window_start 타임스탬프를 추가.

        Args:
        pruned_anomalies (ndarray): 윈도우 내 모든 이상의 시작, 끝 및 max_error를 포함하는 이상 배열.
        errors (ndarray): 오류 배열.
        threshold (float): 임계값.
        window_start (int): 윈도우에서 첫 번째 오류 값의 인덱스.

        Returns:
        list: 각 이상에 대해 시작 인덱스, 종료 인덱스, 점수를 포함하는 이상 목록.
        """
        anomalies = list()

        # 점수 계산을 위한 분모. 오류의 평균과 표준편차의 합.
        denominator = errors.mean() + errors.std()

        # 가지치기된 각 이상에 대해 점수를 계산.
        for row in pruned_anomalies:
            max_error = row[2]

            # 점수를 계산. (max_error - threshold) / 분모
            score = (max_error - threshold) / denominator

            # 절대 인덱스를 사용하여 이상을 추가.
            anomalies.append([row[0] + window_start, row[1] + window_start, score])

        # 이상 목록을 반환.
        return anomalies

In [5]:
errors = [1, 5, 7, 8]
epsilon = [3, 4, 5, 6]
errors > epsilon

False